In [1]:
import os, sys
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#import the functions
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import rtcw_functions as RTCW
reload(RTCW)

# db_name = 'cb.db'
# demos_folder = 'demoscb'

db_name = 'cb.db'
demos_folder = 'demoscb'

cwd = os.getcwd()
demos_path = os.path.join(cwd, '../', demos_folder)

exe_name = 'Anders.Gaming.LibTech3.exe'
root_path = 'C:\\Users\\Jelle\\Documents\\GIT\\rtcw_demo_analyzer' #for executable we need a direct path

In [2]:
#get demos_dct
demos_dct = RTCW.make_dictionary(demos_path)

parsed 100 matches
finished parsing all matches!


In [3]:
#load obituaries in a df
import sqlite3_connector
db = sqlite3_connector.sqlite3_connector(os.path.join(root_path, db_name))

In [4]:
obituary_df = db.get_table_as_df(table_name= 'obituary')
demo_df = db.get_table_as_df(table_name= 'demo')
player_df = db.get_table_as_df(table_name= 'player')
obituary_df = RTCW.add_match_data(obituary_df, player_df, demos_dct)
chatmessages_df = db.get_table_as_df(table_name= 'chatmessage')
chatmessages_df = RTCW.add_match_data(chatmessages_df, player_df, demos_dct, what_df = 'chatmessages_df')
chatmessages_df = RTCW.feature_extraction_chat(chatmessages_df)

obituary table fetched in 0.18 seconds
demo table fetched in 0.0 seconds
player table fetched in 0.02 seconds
chatmessage table fetched in 0.23 seconds


In [5]:
obituary_df = obituary_df[obituary_df['matchName'] == 'rtcw_2003.06.01_cb-ec7_rewind_vs_snaps_finals_sc']

In [6]:
obituary_df = obituary_df[obituary_df['szCleanName'] == '[<<] DeadMeat'] 

In [7]:
obituary_df = obituary_df.iloc[[-1]]

In [8]:
df_spree = RTCW.get_kill_sprees(obituary_df, 
                                demo_df,
                                maxtime_secs = 4, 
                                include_weapon_filter = None,
                                exclude_weapon_filter = None, 
                                minspree = 1,
                                pov_sprees_only = False,
                                verbose = True)

all done!


In [9]:
df_spree[['match', 'demo', 'player', 'spreecount', 'weapons']]

,match,demo,player,spreecount,weapons
0,rtcw_2003.06.01_cb-ec7_rewind_vs_snaps_finals_sc,3.dm_60,[]_Deadmeat,1,5-


In [12]:
RTCW.cut_demos(root_path, demos_dct, df_spree, offset_start = 20, offset_end = 20, transform_to_dm_60 = True, demo_folder_name = demos_folder)
RTCW.generate_capture_list(df_spree, transform_to_dm_60 = True)

In [ ]:
res = RTCW.get_docruns(chatmessages_df, 
                       min_docrun_length = 10, 
                       max_timeleft = 30, 
                       docs_succesful = None, 
                       min_docs_lost = 2, 
                       verbose = True)
res[['start', 'end', 'start_secsleft', 'end_secsleft', 'duration', 'times_lost_docs', 'won_round', 'md5']]

In [11]:
RTCW.cut_demos(root_path, demos_dct, res, demo_type = 'docs', offset_start = 8, offset_end = 8, transform_to_dm_60 = True, demo_folder_name = demos_folder)
RTCW.generate_capture_list(res, demo_type='docs', transform_to_dm_60 = True)

NameError: name 'res' is not defined

In [ ]:
df_wtv = RTCW.get_wtvmoments(chatmessages_df, z = 7, window = 10, verbose=True)
df_wtv

In [ ]:
RTCW.cut_demos(root_path, demos_dct, df_wtv, demo_type = 'wtv', offset_start = 15, offset_end = 8, transform_to_dm_60 = True, demo_folder_name= 'demos')
RTCW.generate_capture_list(df_wtv, demo_type='wtv', transform_to_dm_60 = True)